In [137]:
import pandas as pd
import numpy as np
import math
import datetime
from Tools import *

In [80]:
def format_time(dfTime):
    x=[]
    for i in dfTime:
        i=str(i)
        new_i=""
        count=1
        for j in i:
            if count%2==0 and count!=len(i):
                new_i+=j
                new_i+=":"
            else:
                new_i+=j
            count+=1
        x.append(new_i)
    xy=pd.Series(x)
    return xy

In [81]:
def add_minute(df):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    minutes=[]

    for i in df:
        c=pd.to_datetime(i)
        after6M=c+relativedelta(minutes=+6)
        befor6M=c+relativedelta(minutes=-6)
        minutes.append([after6M,befor6M,i])
    return minutes

# Dosyalardaki verileri tek dosyada toplama- Bu kısım tek seferlik. Silinecek

In [82]:
df_news=pd.DataFrame()
for i in range(1,7):
    path="TeslaNews"+str(i)+".csv"
    temp_df=pd.read_csv(path)
    df_news=df_news.append(temp_df)

In [83]:
df_news.info()
df_news.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9536 entries, 0 to 1586
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9536 non-null   int64 
 1   Dates       9536 non-null   object
 2   Text        9536 non-null   object
 3   Link        9536 non-null   object
dtypes: int64(1), object(3)
memory usage: 372.5+ KB


,Unnamed: 0,Dates,Text,Link
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...


In [84]:
df_news.drop(df_news.columns[df_news.columns.str.contains("Unnamed",case=False)],axis=1,inplace=True)

In [85]:

df_news[df_news.duplicated(keep=False)]
df_news.insert(0,"ID",range(0,len(df_news)))

Format Time columns

In [86]:
df=pd.read_csv("tsla.us.txt",sep=",",names=['TICKER','PER','DATE','TIME','OPEN','HIGH','LOW','CLOSE','VOL','OPENINT'])
df=df.drop(["PER","TICKER"],axis='columns')
df["DATE2"]=pd.to_datetime(df["DATE"],format="%Y%m%d")
df["DATE2"]=df["DATE2"].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y"))
df["TIME2"]=format_time(df["TIME"])
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00
...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00


In [87]:
df["MOVE"]=df["OPEN"]-df["CLOSE"]
df.loc[df['MOVE'] < 0, 'STATUS'] = -1
df.loc[df['MOVE'] > 0, 'STATUS'] = 1
df.loc[df['MOVE'] == 0, 'STATUS'] = 0
df["PERC"]=(df["MOVE"]*100)/df["OPEN"]
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2,MOVE,STATUS,PERC
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00,-2.419,-1.0,-0.544072
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00,4.880,1.0,1.091625
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00,-10.069,-1.0,-2.277642
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00,-7.568,-1.0,-1.674929
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00,2.790,1.0,0.607578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00,-1.055,-1.0,-0.244309
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00,0.880,1.0,0.203299
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00,-1.950,-1.0,-0.451389
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00,1.350,1.0,0.311096


In [88]:
df["CCCC"]=df["DATE2"]+" "+df["TIME2"]
df["CCCC"]=pd.to_datetime(df["CCCC"])
df["CCCC"]=df["CCCC"].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y %H:%M:%S"))
df["CCCC"]=pd.to_datetime(df["CCCC"])
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2,MOVE,STATUS,PERC,CCCC
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00,-2.419,-1.0,-0.544072,2020-08-31 15:30:00
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00,4.880,1.0,1.091625,2020-08-31 15:35:00
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00,-10.069,-1.0,-2.277642,2020-08-31 15:40:00
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00,-7.568,-1.0,-1.674929,2020-08-31 15:45:00
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00,2.790,1.0,0.607578,2020-08-31 15:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00,-1.055,-1.0,-0.244309,2020-10-19 21:35:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00,0.880,1.0,0.203299,2020-10-19 21:40:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00,-1.950,-1.0,-0.451389,2020-10-19 21:45:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00,1.350,1.0,0.311096,2020-10-19 21:50:00


In [89]:
df_news['NewDates'] = pd.to_datetime(df_news['Dates'])
#df_news['NewDates'] = df_news['NewDates'].dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
df_news['NewDates'] = df_news['NewDates'].apply(lambda x: datetime.datetime.replace(x, tzinfo=None))
df_news

C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,ID,Dates,Text,Link,NewDates
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 07:41:00
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 07:11:00
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 07:06:00
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 06:22:00
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 05:28:00
...,...,...,...,...,...
1582,9531,"Apr 29, 2016 3:43AM EDT",Tesla CEO Elon Musk introduces the Model X. So...,https://www.nasdaq.com/articles/mark-zuckerber...,2016-04-29 03:43:00
1583,9532,"Apr 29, 2016 2:22AM EDT",Ford ( F ) reported an all time record pre-tax...,https://www.nasdaq.com/articles/ford-posts-rec...,2016-04-29 02:22:00
1584,9533,"Apr 28, 2016 11:44AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/alphabet-inc%3...,2016-04-28 11:44:00
1585,9534,"Apr 28, 2016 10:23AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/apple-inc.-aap...,2016-04-28 10:23:00


In [51]:

df_news

,Dates,Text,Link,NewDates
0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 07:41:00
1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 07:11:00
2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 07:06:00
3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 06:22:00
4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 05:28:00
...,...,...,...,...
1582,"Apr 29, 2016 3:43AM EDT",Tesla CEO Elon Musk introduces the Model X. So...,https://www.nasdaq.com/articles/mark-zuckerber...,2016-04-29 03:43:00
1583,"Apr 29, 2016 2:22AM EDT",Ford ( F ) reported an all time record pre-tax...,https://www.nasdaq.com/articles/ford-posts-rec...,2016-04-29 02:22:00
1584,"Apr 28, 2016 11:44AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/alphabet-inc%3...,2016-04-28 11:44:00
1585,"Apr 28, 2016 10:23AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/apple-inc.-aap...,2016-04-28 10:23:00


In [90]:
#df_news["Dates"]=datetime.strptime(df_news["Dates"],"%d/%m/%Y")
#df_news['NewDates'] = df_news['NewDates'].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y %H:%M:%S"))


df_temp2=df_news[df_news.duplicated(keep=False)]
df_temp=df_news[df_news.duplicated(subset=["NewDates"],keep=False)]
df_temp=df_temp.drop(["Link"],axis=1)
df_temp2
df_news

,ID,Dates,Text,Link,NewDates
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 07:41:00
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 07:11:00
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 07:06:00
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 06:22:00
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 05:28:00
...,...,...,...,...,...
1582,9531,"Apr 29, 2016 3:43AM EDT",Tesla CEO Elon Musk introduces the Model X. So...,https://www.nasdaq.com/articles/mark-zuckerber...,2016-04-29 03:43:00
1583,9532,"Apr 29, 2016 2:22AM EDT",Ford ( F ) reported an all time record pre-tax...,https://www.nasdaq.com/articles/ford-posts-rec...,2016-04-29 02:22:00
1584,9533,"Apr 28, 2016 11:44AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/alphabet-inc%3...,2016-04-28 11:44:00
1585,9534,"Apr 28, 2016 10:23AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/apple-inc.-aap...,2016-04-28 10:23:00


In [93]:
df_news[df_news.duplicated(subset=["Text"],keep=False)]

,ID,Dates,Text,Link,NewDates
62,62,"Oct 6, 2020 9:10AM EDT","InvestorPlace - Stock Market News, Stock Advic...",https://www.nasdaq.com/articles/how-to-invest-...,2020-10-06 09:10:00
81,81,"Oct 4, 2020 9:00AM EDT","By David RandallNEW YORK, Oct 2 (Reuters) - In...",https://www.nasdaq.com/articles/wall-st-week-a...,2020-10-04 09:00:00
89,89,"Oct 2, 2020 4:52PM EDT","By David RandallNEW YORK, Oct 2 (Reuters) - In...",https://www.nasdaq.com/articles/wall-st-week-a...,2020-10-02 16:52:00
163,163,"Sep 29, 2020 2:00AM EDT","By Marc JonesLONDON, Sept 28 (Reuters) - Finan...",https://www.nasdaq.com/articles/graphic-market...,2020-09-29 02:00:00
166,166,"Sep 28, 2020 7:00PM EDT","By Marc JonesLONDON, Sept 28 (Reuters) - Finan...",https://www.nasdaq.com/articles/graphic-market...,2020-09-28 19:00:00
...,...,...,...,...,...
90,8039,"Feb 28, 2017 7:31AM EST","The stock market is, for lack of a better word...",https://www.nasdaq.com/articles/its-official-t...,2017-02-28 07:31:00
123,8072,"Feb 24, 2017 12:12PM EST",Shutterstock photoThe broad stock market outpa...,https://www.nasdaq.com/articles/jpmorgan-fund-...,2017-02-24 12:12:00
124,8073,"Feb 24, 2017 12:12PM EST",Shutterstock photoThe broad stock market outpa...,https://www.nasdaq.com/articles/jpmorgan-fund-...,2017-02-24 12:12:00
575,8524,"Nov 17, 2016 2:14AM EST","According to Trefis estimates, nearly half of ...",https://www.nasdaq.com/articles/how-fast-must-...,2016-11-17 02:14:00


In [128]:
d=pd.read_csv("links.txt",names=["links"],header=None,)

In [129]:
d.shape

(9600, 1)

In [146]:
file1 = r'C:\Users\enesy\OneDrive\Okul\Tez\Codes\tez\getfinnews\links.txt'
#Lines=file1.readlines()
Lines=File.ReadFromFile(file1)
bulk_size=len(xNews)/2000
bulk_size=round(bulk_size)
newsLinks=[]
news=np.array(newsLinks)
xNews=np.array(news).flatten()
 

print(bulk_size)
while i<=bulk_size
for i in range(0,len(xNews)):
    newsLinks.append(xNews[i])
    

    

5


In [130]:
f=d.drop_duplicates(subset=["links"])
d=d.drop_duplicates(subset=["links"])
print(d.shape)
f
d[d.duplicated(subset=["links"],keep=False)]

(9595, 1)


,links


In [92]:


minute_range=add_minute(df_news["NewDates"])
y=pd.DataFrame(minute_range,columns=["After6M","Before6M","Original"])
f=pd.merge(df_news,y,on="NewDates")
f

KeyError: 'NewDates'

In [104]:
df_news.head()

,ID,Unnamed: 0,Dates,Text,Link,NewDates
0,0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 11:41:00
1,1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 11:11:00
2,2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 11:06:00
3,3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 10:22:00
4,4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 09:28:00


In [80]:
df_price=pd.read_csv("TSLA.csv")

In [88]:
pd.concat([df_news,df_price],axis=1,join="inner")

,Unnamed: 0,Dates,Text,Link,NewDates,Date,Open,High,Low,Close,Adj Close,Volume
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,13/10/2020,12/10/2010,4.040000,4.056000,4.006000,4.048000,4.048000,1220000
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,13/10/2020,13/10/2010,4.128000,4.170000,4.072000,4.108000,4.108000,1591000
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,13/10/2020,14/10/2010,4.200000,4.206000,4.080000,4.150000,4.150000,1474000
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,13/10/2020,15/10/2010,4.178000,4.180000,4.050000,4.108000,4.108000,1423500
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,13/10/2020,18/10/2010,4.104000,4.128000,4.044000,4.046000,4.046000,814000
...,...,...,...,...,...,...,...,...,...,...,...,...
966,966,"Aug 11, 2020 2:07PM EDT","InvestorPlace - Stock Market News, Stock Advic...",https://www.nasdaq.com/articles/electrameccani...,11/08/2020,14/08/2014,52.498001,52.599998,51.299999,52.276001,52.276001,20633000
967,967,"Aug 11, 2020 12:46PM EDT",Aspiring luxury electric vehicle (EV) maker Lu...,https://www.nasdaq.com/articles/lucid-motors-s...,11/08/2020,15/08/2014,52.296001,52.417999,51.700001,52.402000,52.402000,19339500
968,968,"Aug 11, 2020 11:10AM EDT","Adds executive quotes, updates share price mov...",https://www.nasdaq.com/articles/chinese-ev-mak...,11/08/2020,18/08/2014,52.650002,53.452000,51.950001,51.987999,51.987999,29246000
969,969,"Aug 11, 2020 10:52AM EDT","InvestorPlace - Stock Market News, Stock Advic...",https://www.nasdaq.com/articles/roku-stock-buy...,11/08/2020,19/08/2014,51.773998,51.866001,50.324001,51.352001,51.352001,26674000
